In [57]:
from azureml.core import Workspace, Datastore, Experiment
ws = Workspace('bd04922c-a444-43dc-892f-74d5090f8a9a','mlplayarearg','testdeployment')

In [58]:
mydatastrore1 = Datastore.get(ws, 'gen2datastoreasblob')
mydatastrore2 = Datastore.get(ws, 'gen2datastoreasblob2')

In [59]:
from azureml.data.datapath import DataPath, DataPathComputeBinding
from  azureml.pipeline.core.graph import PipelineParameter
data_path1 = DataPath(datastore=mydatastrore1, path_on_datastore='somedata')
datapath1_pipeline_param = PipelineParameter(name="input_datapath1", default_value=data_path1)
datapath_input1 = (datapath1_pipeline_param, DataPathComputeBinding(mode='mount'))

data_path2 = DataPath(datastore=mydatastrore2, path_on_datastore='somedata')
datapath2_pipeline_param = PipelineParameter(name="input_datapath2", default_value=data_path2)
datapath_input2 = (datapath2_pipeline_param, DataPathComputeBinding(mode='mount'))

string_pipeline_param = PipelineParameter(name="input_string", default_value='sample_string1')

In [60]:

%%writefile step1.py
import argparse
import os
import pandas as pd

parser = argparse.ArgumentParser("train")
parser.add_argument("--arg1", type=str, help="sample string argument")
parser.add_argument("--arg2", type=str, help="sample datapath argument")
args = parser.parse_args()




mydf1 = pd.read_csv(os.path.join(args.arg2,'mydata.csv'))

mydf2 = pd.read_csv(os.path.join(args.arg2,'testdataatroot.csv'))
print(mydf1.shape[0])
print(mydf2.shape[0])
numberofrows = mydf1.shape[0] + mydf2.shape[0]

test = pd.DataFrame(range(1,numberofrows), columns=["testColumn"])
for x in range(6):
    if not os.path.exists(os.path.join(args.arg2,args.arg1)):
        print (os.path.join(args.arg2,args.arg1))
        os.makedirs(os.path.join(args.arg2,args.arg1))
    csvfilepath = os.path.join(args.arg2,args.arg1 ,'file' + str(x) + '.csv')
    test.to_csv(csvfilepath)

print("Sample string argument  : %s" % args.arg1)
print("Sample datapath argument: %s" % args.arg2)

Overwriting step1.py


In [61]:
%%writefile step2.py
import argparse
import os
import pandas as pd

parser = argparse.ArgumentParser("train")
parser.add_argument("--arg1", type=str, help="sample string argument")
parser.add_argument("--arg2", type=str, help="sample datapath argument")
args = parser.parse_args()




mydf1 = pd.read_csv(os.path.join(args.arg2,'mydata.csv'))

mydf2 = pd.read_csv(os.path.join(args.arg2,'testdataatroot.csv'))
print(mydf1.shape[0])
print(mydf2.shape[0])
numberofrows = mydf1.shape[0] + mydf2.shape[0]

test = pd.DataFrame(range(1,numberofrows), columns=["testColumn"])
for x in range(6):
    if not os.path.exists(os.path.join(args.arg2,args.arg1)):
        print (os.path.join(args.arg2,args.arg1))
        os.makedirs(os.path.join(args.arg2,args.arg1), exist_ok = True)
    csvfilepath = os.path.join(args.arg2,args.arg1 ,'file' + str(x) + '.csv')
    test.to_csv(csvfilepath)

print("Sample string argument  : %s" % args.arg1)
print("Sample datapath argument: %s" % args.arg2)

Overwriting step2.py


In [62]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 

aml_run_config = RunConfiguration()
# `compute_target` as defined in "Azure Machine Learning compute" section above
aml_run_config.target = "cpu-cluster"

USE_CURATED_ENV = False
if USE_CURATED_ENV :
    curated_environment = Environment.get(workspace=ws, name="AzureML-Tutorial")
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies = False
    
    # Add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=['pandas','scikit-learn'], 
        pip_packages=['azureml-sdk', 'azureml-dataprep[fuse,pandas]'], 
        pin_sdk_version=False)
    

In [35]:
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.core import Pipeline, PipelineRun
from azureml.pipeline.steps import PythonScriptStep

train_step1 = PythonScriptStep(
    name='train_step1',
    script_name="step1.py",
    arguments=["--arg1", string_pipeline_param, "--arg2", datapath_input1],
    inputs=[datapath_input1],
    runconfig=aml_run_config,
    source_directory='.')
print("train_step1 created")

train_step2 = PythonScriptStep(
    name='train_step2',
    script_name="step2.py",
    arguments=["--arg1", string_pipeline_param, "--arg2", datapath_input2],
    inputs=[datapath_input2],
    runconfig=aml_run_config,
    source_directory='.')
print("train_step2 created")

pipeline = Pipeline(workspace=ws, steps=[train_step1,train_step2])
print("pipeline with the train_steps created")

experiment_name = 'showcasing-datapath'
source_directory  = '.'

experiment = Experiment(ws, experiment_name)
newdatastore = Datastore.get(ws,'gen2datastoreasblob')
new_data_path1 = DataPath(datastore=newdatastore, path_on_datastore='somedata')
newdatastore2 = Datastore.get(ws,'gen2datastoreasblob2')
new_data_path2 = DataPath(datastore=newdatastore2, path_on_datastore='somedata')

pipeline_run = experiment.submit(pipeline,pipeline_parameters={"input_string": "20201127110800","input_datapath1":new_data_path1,"input_datapath2":new_data_path2})

train_step1 created
train_step2 created
pipeline with the train_steps created
Created step train_step1 [53101c75][1bab8338-4062-4a09-b70c-49e887fc0060], (This step will run and generate new outputs)
Created step train_step2 [f54932d5][a9834ae1-f324-4ce4-8373-6d1df9aeb50d], (This step will run and generate new outputs)
Using data reference newdatastoremanish_19178a81 for StepId [7a970874][22522887-bdc7-4255-9de9-91ff73f6dfcb], (Consumers of this data are eligible to reuse prior runs.)
Using data reference newdatastoremanish_19178a81 for StepId [b4247355][22522887-bdc7-4255-9de9-91ff73f6dfcb], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun a9bc1662-9705-4a23-a3ae-8d4e9486ee8e
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/showcasing-datapath/runs/a9bc1662-9705-4a23-a3ae-8d4e9486ee8e?wsid=/subscriptions/bd04922c-a444-43dc-892f-74d5090f8a9a/resourcegroups/mlplayarearg/workspaces/testdeployment


In [55]:
datapath_input2

(<azureml.pipeline.core.graph.PipelineParameter at 0x7ff5fc9d9fd0>,
 <azureml.data.datapath.DataPathComputeBinding at 0x7ff5fc9d9f28>)

In [63]:
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.core import Pipeline, PipelineRun
from azureml.pipeline.steps import PythonScriptStep

train_step1 = PythonScriptStep(
    name='train_step1',
    script_name="step1.py",
    arguments=["--arg1", string_pipeline_param, "--arg2", datapath_input1],
    inputs=[datapath_input1],
    runconfig=aml_run_config,
    source_directory='.')
print("train_step1 created")

train_step2 = PythonScriptStep(
    name='train_step2',
    script_name="step2.py",
    arguments=["--arg1", string_pipeline_param, "--arg2", datapath_input2],
    inputs=[datapath_input2],
    runconfig=aml_run_config,
    source_directory='.')
print("train_step2 created")

pipeline = Pipeline(workspace=ws, steps=[train_step1,train_step2])
print("pipeline with the train_steps created")


pipeline.publish(name='PublishedForTriggerlater',description='this is awsesome pipeline',version='3.0')

train_step1 created
train_step2 created
pipeline with the train_steps created
Created step train_step1 [3ff0c312][8fa93624-848e-45de-a15d-c1137cdc2842], (This step will run and generate new outputs)
Created step train_step2 [92c03f92][27687da4-2d7c-4682-91a2-aa8e89bfb405], (This step will run and generate new outputs)
Created data reference gen2datastoreasblob_751c8811 for StepId [01dbc3d8][61593f49-1560-4dbb-a321-c0e18c582bc1], (Consumers of this data will generate new runs.)
Created data reference gen2datastoreasblob2_981d8e0d for StepId [a138a5f0][4a762f56-980e-445f-8e53-90e49c3d7334], (Consumers of this data will generate new runs.)


Name,Id,Status,Endpoint
PublishedForTriggerlater,9f116f0d-2d43-4f1e-bfab-33400de1b1f2,Active,REST Endpoint
